In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 30
learning_rate = 0.001
batch_size = 100

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                                train=True,
                                                transform=transform,
                                                download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                                  train=False,
                                                    transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                  batch_size=batch_size,
                                                    shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                    shuffle=False)    

Files already downloaded and verified


![resnetimage](https://user-images.githubusercontent.com/30661597/78585170-f4ac7c80-786b-11ea-8b00-8b751c65f5ca.PNG)

In [3]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [4]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.downsample = downsample

    def forward(self, x):
        residual = x
        
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.downsample:
            residual = self.downsample(x)
        
        out += residual
        out = self.relu(out)
        return out

In [5]:
#define ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        
        """"
        layers will be a list of 3 numbers, each number represents the number of residual blocks in each layer
        [2,2,2]
        """
        super(ResNet, self).__init__()
        
        self.in_channels = 16
        
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsampling = None
        if (self.in_channels != out_channels) or (stride != 1):
            downsampling = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
            
        residual_blocks = []
        residual_blocks.append(block(self.in_channels, out_channels, stride, downsampling))
        self.in_channels = out_channels
        residual_blocks.append(block(self.in_channels, out_channels))
        return nn.Sequential(*residual_blocks)
    
    def forward(self, x):
        out = self.relu(self.bn(self.conv(x)))
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [6]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
decay = 0
for epoch in range(num_epochs):
    # Decay learning rate by a factor of 0.5 every 20 epochs
    if (epoch+1) % 20 == 0:
        decay += 1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("Learning rate is set to {}".format(optimizer.param_groups[0]['lr']))

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print("Epoch [{}/{}], Iter [{}/{}] Loss: {:.4f}"
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))


Epoch [1/30], Iter [100/500] Loss: 1.4511
Epoch [1/30], Iter [200/500] Loss: 1.4091
Epoch [1/30], Iter [300/500] Loss: 1.4517
Epoch [1/30], Iter [400/500] Loss: 1.2435
Epoch [1/30], Iter [500/500] Loss: 1.0508
Epoch [2/30], Iter [100/500] Loss: 1.0552
Epoch [2/30], Iter [200/500] Loss: 1.1218
Epoch [2/30], Iter [300/500] Loss: 0.9699
Epoch [2/30], Iter [400/500] Loss: 0.9952
Epoch [2/30], Iter [500/500] Loss: 1.0469
Epoch [3/30], Iter [100/500] Loss: 0.7809
Epoch [3/30], Iter [200/500] Loss: 0.8516
Epoch [3/30], Iter [300/500] Loss: 0.8789
Epoch [3/30], Iter [400/500] Loss: 0.7062
Epoch [3/30], Iter [500/500] Loss: 0.7301
Epoch [4/30], Iter [100/500] Loss: 0.7195
Epoch [4/30], Iter [200/500] Loss: 0.9060
Epoch [4/30], Iter [300/500] Loss: 0.7624
Epoch [4/30], Iter [400/500] Loss: 0.6753
Epoch [4/30], Iter [500/500] Loss: 0.5646
Epoch [5/30], Iter [100/500] Loss: 0.8643
Epoch [5/30], Iter [200/500] Loss: 0.8689
Epoch [5/30], Iter [300/500] Loss: 0.7598
Epoch [5/30], Iter [400/500] Loss:

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [9]:
optimizer.param_groups[0]['lr']

0.001